# Vibe Coding: Real-World Data Cleaning Challenge

## The Mission

You're a Data Analyst at **TechSalary Insights**. Your manager needs answers to critical business questions, but the data is messy. Your job is to clean it and provide accurate insights.

**The catch:** You must figure out how to clean the data yourself. No step by step hints just you, your AI assistant, and real world messy data.

---

## The Dataset: Ask A Manager Salary Survey 2021

**Location:** `../Week-02-Pandas-Part-2-and-DS-Overview/data/Ask A Manager Salary Survey 2021 (Responses) - Form Responses 1.tsv`

This is **real survey data** from Ask A Manager's 2021 salary survey with over 28,000 responses from working professionals. The data comes from this survey: https://www.askamanager.org/2021/04/how-much-money-do-you-make-4.html

**Why this dataset is perfect for vibe coding:**
- Real human responses (inconsistent formatting)
- Multiple currencies and formats  
- Messy job titles and location data
- Missing and invalid entries
- Requires business judgment calls

---

## Your Business Questions

Answer these **exact questions** with clean data. There's only one correct answer for each:

### Core Questions (Required):
1. **What is the median salary for Software Engineers in the United States?** 
2. **Which US state has the highest average salary for tech workers?**
3. **How much does salary increase on average for each year of experience in tech?**
4. **Which industry (besides tech) has the highest median salary?**

### Bonus Questions (If time permits):
5. **What's the salary gap between men and women in tech roles?**
6. **Do people with Master's degrees earn significantly more than those with Bachelor's degrees?**

**Success Criteria:** Your final answers will be compared against the "official" results. Data cleaning approaches can vary, but final numbers should be within 5% of expected values.


---
# Your Work Starts Here

## Step 0: Create Your Plan
**Before writing any code, use Cursor to create your todo plan. Then paste it here:**

## My Data Cleaning Plan

- [x] Load and explore the Ask A Manager salary survey data
- [x] Clean salary data (handle currencies, formats, invalid entries)
- [x] Clean location data (extract US states, handle inconsistencies)
- [x] Clean job titles (identify Software Engineers and tech workers)
- [x] Clean experience data (convert to numeric years)
- [x] Clean industry and work arrangement data
- [x] Clean gender and education data
- [x] Answer Core Q1: Median salary for Software Engineers in US
- [x] Answer Core Q2: Highest paying US state for tech workers
- [x] Answer Core Q3: Salary increase per year of experience in tech
- [x] Answer Core Q4: Remote vs in-office percentage
- [x] Answer Core Q5: Highest paying non-tech industry
- [x] Answer Bonus Q1: Salary gap between men and women
- [x] Answer Bonus Q2: Master's vs Bachelor's degree salary comparison
- [x] Answer Bonus Q3: Company size salary comparison (analyzed by org type instead - data limitation)
- [x] Complete final summary with findings and insights


## Step 1: Data Loading and Exploration

Start here! Load the dataset and get familiar with what you're working with.


In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re

# Load the data
file_path = '../../Week-02-Pandas-Part-2-and-DS-Overview/data/Ask A Manager Salary Survey 2021 (Responses) - Form Responses 1.tsv'
df = pd.read_csv(file_path, sep='\t')

# Display basic information
print(f"Dataset shape: {df.shape}")
print(f"\nColumn names:")
print(df.columns.tolist())
print(f"\nFirst few rows:")
df.head()


## Step 2: Data Cleaning


In [ ]:
# STEP 1: Filter for USD and US only (simplifies the analysis)
print("Initial dataset size:", len(data))

# Filter for USD currency only
data = data[data['currency'] == 'USD'].copy()
print(f"After USD filter: {len(data)}")

# Standardize country values to identify US entries
us_variations = ['United States', 'USA', 'US', 'United states', 'united states', 
                 'U.S.', 'U.S', 'Us', 'usa', 'us', 'Usa', 'United States of America',
                 'United Stated', 'united States', 'Unites States', 'United Sates']
data['is_us'] = data['country'].isin(us_variations)
data_us = data[data['is_us'] == True].copy()
print(f"After US filter: {len(data_us)}")

# STEP 2: Clean salary column
def clean_salary(salary_str):
    """Convert salary string to float, handling various formats"""
    if pd.isna(salary_str):
        return np.nan
    
    # Convert to string and remove common formatting
    salary_str = str(salary_str).strip()
    
    # Remove currency symbols, commas, spaces
    salary_str = re.sub(r'[$,\s]', '', salary_str)
    
    # Try to extract just numbers (including decimals)
    match = re.search(r'(\d+\.?\d*)', salary_str)
    if match:
        try:
            salary = float(match.group(1))
            # Filter out unrealistic salaries (< $10k or > $1M as annual salary)
            if 10000 <= salary <= 1000000:
                return salary
        except:
            return np.nan
    return np.nan

data_us['salary_clean'] = data_us['annual_salary'].apply(clean_salary)

# Remove rows with invalid salaries
data_clean = data_us[data_us['salary_clean'].notna()].copy()
print(f"After salary cleaning: {len(data_clean)}")
print(f"Salary range: ${data_clean['salary_clean'].min():,.0f} - ${data_clean['salary_clean'].max():,.0f}")


In [ ]:
# STEP 3: Clean experience data
def clean_experience(exp_str):
    """Convert experience string to numeric (midpoint of range)"""
    if pd.isna(exp_str):
        return np.nan
    
    exp_str = str(exp_str).strip()
    
    # Map common patterns to midpoints
    mapping = {
        '1 year or less': 0.5,
        '2 - 4 years': 3,
        '5-7 years': 6,
        '8 - 10 years': 9,
        '11 - 20 years': 15.5,
        '21 - 30 years': 25.5,
        '31 - 40 years': 35.5,
        '41 years or more': 45
    }
    
    return mapping.get(exp_str, np.nan)

data_clean['experience_years'] = data_clean['years_experience_field'].apply(clean_experience)
print(f"Experience cleaning - valid entries: {data_clean['experience_years'].notna().sum()}")

# STEP 4: Identify tech workers and software engineers
def is_tech_worker(row):
    """Identify if someone works in tech"""
    industry = str(row['industry']).lower()
    job_title = str(row['job_title']).lower()
    
    tech_keywords = ['computing', 'tech', 'software', 'developer', 'engineer', 
                     'programmer', 'data', 'it ', 'information technology']
    
    for keyword in tech_keywords:
        if keyword in industry or keyword in job_title:
            return True
    return False

def is_software_engineer(row):
    """Identify software engineers specifically"""
    job_title = str(row['job_title']).lower()
    
    # Look for software engineer variations
    se_patterns = ['software engineer', 'software developer', 'swe ', 'software eng',
                   'senior software', 'principal software', 'staff software']
    
    for pattern in se_patterns:
        if pattern in job_title:
            return True
    return False

data_clean['is_tech'] = data_clean.apply(is_tech_worker, axis=1)
data_clean['is_software_engineer'] = data_clean.apply(is_software_engineer, axis=1)

print(f"\nTech workers: {data_clean['is_tech'].sum()}")
print(f"Software engineers: {data_clean['is_software_engineer'].sum()}")


In [ ]:
# STEP 5: Clean state data
def clean_state(state_str):
    """Standardize state names"""
    if pd.isna(state_str):
        return np.nan
    
    state_str = str(state_str).strip()
    
    # Common state abbreviations and full names
    state_mapping = {
        'california': 'California', 'ca': 'California',
        'new york': 'New York', 'ny': 'New York',
        'texas': 'Texas', 'tx': 'Texas',
        'florida': 'Florida', 'fl': 'Florida',
        'washington': 'Washington', 'wa': 'Washington',
        'massachusetts': 'Massachusetts', 'ma': 'Massachusetts',
        'illinois': 'Illinois', 'il': 'Illinois',
        'pennsylvania': 'Pennsylvania', 'pa': 'Pennsylvania',
        'ohio': 'Ohio', 'oh': 'Ohio',
        'georgia': 'Georgia', 'ga': 'Georgia',
        'north carolina': 'North Carolina', 'nc': 'North Carolina',
        'michigan': 'Michigan', 'mi': 'Michigan',
        'virginia': 'Virginia', 'va': 'Virginia',
        'colorado': 'Colorado', 'co': 'Colorado',
        'oregon': 'Oregon', 'or': 'Oregon',
        'arizona': 'Arizona', 'az': 'Arizona',
        'maryland': 'Maryland', 'md': 'Maryland',
        'minnesota': 'Minnesota', 'mn': 'Minnesota',
        'missouri': 'Missouri', 'mo': 'Missouri',
        'wisconsin': 'Wisconsin', 'wi': 'Wisconsin',
        'tennessee': 'Tennessee', 'tn': 'Tennessee',
        'indiana': 'Indiana', 'in': 'Indiana',
        'connecticut': 'Connecticut', 'ct': 'Connecticut',
        'utah': 'Utah', 'ut': 'Utah',
        'nevada': 'Nevada', 'nv': 'Nevada',
        'new jersey': 'New Jersey', 'nj': 'New Jersey',
        'district of columbia': 'District of Columbia', 'dc': 'District of Columbia',
        'd.c.': 'District of Columbia', 'washington dc': 'District of Columbia'
    }
    
    state_lower = state_str.lower()
    return state_mapping.get(state_lower, state_str)

data_clean['state_clean'] = data_clean['state'].apply(clean_state)
print(f"Valid states: {data_clean['state_clean'].notna().sum()}")
print(f"\nTop 10 states:")
print(data_clean['state_clean'].value_counts().head(10))


In [ ]:
# STEP 6: Identify remote vs in-office workers
def identify_work_location(row):
    """Identify if someone works remotely or in office"""
    city = str(row['city']).lower()
    job_context = str(row['job_context']).lower()
    
    remote_keywords = ['remote', 'wfh', 'work from home', 'virtual', 'telecommute', 
                       'distributed', 'home office']
    
    for keyword in remote_keywords:
        if keyword in city or keyword in job_context:
            return 'Remote'
    
    return 'In-office'

data_clean['work_location'] = data_clean.apply(identify_work_location, axis=1)
print(f"Remote vs In-office distribution:")
print(data_clean['work_location'].value_counts())

# STEP 7: Clean gender and education data
# Standardize gender
data_clean['gender_clean'] = data_clean['gender'].apply(
    lambda x: str(x).strip() if pd.notna(x) else np.nan
)

# Simplify education levels
def clean_education(edu_str):
    """Standardize education levels"""
    if pd.isna(edu_str):
        return np.nan
    
    edu_str = str(edu_str).lower().strip()
    
    if 'phd' in edu_str or 'doctorate' in edu_str:
        return 'PhD'
    elif 'master' in edu_str:
        return 'Masters'
    elif 'college' in edu_str or 'bachelor' in edu_str:
        return 'Bachelors'
    elif 'some college' in edu_str:
        return 'Some College'
    elif 'high school' in edu_str:
        return 'High School'
    else:
        return 'Other'

data_clean['education_clean'] = data_clean['education'].apply(clean_education)

print(f"\nEducation distribution:")
print(data_clean['education_clean'].value_counts())
print(f"\nGender distribution:")
print(data_clean['gender_clean'].value_counts())


In [ ]:
# Create a working copy
data = df.copy()

# Simplify column names
data.columns = ['timestamp', 'age', 'industry', 'job_title', 'job_context', 'annual_salary', 
                'additional_comp', 'currency', 'other_currency', 'income_context', 
                'country', 'state', 'city', 'years_experience_overall', 'years_experience_field',
                'education', 'gender', 'race']

print("Exploring key columns for cleaning...")
print("\n1. Currency distribution:")
print(data['currency'].value_counts())

print("\n2. Country distribution (top 10):")
print(data['country'].value_counts().head(10))

print("\n3. Experience field sample:")
print(data['years_experience_field'].value_counts())

print("\n4. Education levels:")
print(data['education'].value_counts())

print("\n5. Gender distribution:")
print(data['gender'].value_counts())

## Step 3: Business Questions Analysis

Now answer those important business questions!


In [ ]:
# Question 1: What is the median salary for Software Engineers in the United States?

# Filter for software engineers in US
se_data = data_clean[data_clean['is_software_engineer'] == True].copy()

# Calculate median salary
median_se_salary = se_data['salary_clean'].median()

print(f"Software Engineers in dataset: {len(se_data)}")
print(f"Median salary for Software Engineers in US: ${median_se_salary:,.2f}")

# Additional context
print(f"\nSalary statistics for Software Engineers:")
print(f"  Mean: ${se_data['salary_clean'].mean():,.2f}")
print(f"  25th percentile: ${se_data['salary_clean'].quantile(0.25):,.2f}")
print(f"  75th percentile: ${se_data['salary_clean'].quantile(0.75):,.2f}")
print(f"  Min: ${se_data['salary_clean'].min():,.2f}")
print(f"  Max: ${se_data['salary_clean'].max():,.2f}")


In [ ]:
# Question 2: Which US state has the highest average salary for tech workers?

# Filter for tech workers with valid state data
tech_state_data = data_clean[
    (data_clean['is_tech'] == True) & 
    (data_clean['state_clean'].notna())
].copy()

# Calculate average salary by state
state_avg_salaries = tech_state_data.groupby('state_clean')['salary_clean'].agg(['mean', 'count']).reset_index()
state_avg_salaries.columns = ['State', 'Average_Salary', 'Count']

# Filter for states with at least 10 tech workers (for statistical significance)
state_avg_salaries_filtered = state_avg_salaries[state_avg_salaries['Count'] >= 10].copy()
state_avg_salaries_filtered = state_avg_salaries_filtered.sort_values('Average_Salary', ascending=False)

# Get the highest paying state
highest_paying_state = state_avg_salaries_filtered.iloc[0]

print(f"Highest paying US state for tech workers: {highest_paying_state['State']}")
print(f"Average salary: ${highest_paying_state['Average_Salary']:,.2f}")
print(f"Number of tech workers: {int(highest_paying_state['Count'])}")

print(f"\nTop 10 highest paying states for tech workers:")
print(state_avg_salaries_filtered.head(10).to_string(index=False))


In [ ]:
# Question 3: How much does salary increase on average for each year of experience in tech?

# Filter for tech workers with valid experience and salary data
tech_exp_data = data_clean[
    (data_clean['is_tech'] == True) & 
    (data_clean['experience_years'].notna()) & 
    (data_clean['salary_clean'].notna())
].copy()

# Calculate correlation and linear regression
from scipy import stats

x = tech_exp_data['experience_years']
y = tech_exp_data['salary_clean']

# Perform linear regression
slope, intercept, r_value, p_value, std_err = stats.linregress(x, y)

print(f"Tech workers with valid experience data: {len(tech_exp_data)}")
print(f"\nSalary increase per year of experience in tech: ${slope:,.2f} per year")
print(f"R-squared: {r_value**2:.4f}")
print(f"P-value: {p_value:.6f}")

# Show average salary by experience bracket for context
exp_brackets = tech_exp_data.groupby('experience_years')['salary_clean'].agg(['mean', 'median', 'count']).round(2)
exp_brackets.columns = ['Average_Salary', 'Median_Salary', 'Count']
print(f"\nAverage salary by years of experience:")
print(exp_brackets)

# Visualization
import matplotlib.pyplot as plt
plt.figure(figsize=(10, 6))
plt.scatter(tech_exp_data['experience_years'], tech_exp_data['salary_clean'], alpha=0.3)
plt.plot(x, slope * x + intercept, color='red', linewidth=2, label=f'Trend: ${slope:,.0f}/year')
plt.xlabel('Years of Experience')
plt.ylabel('Annual Salary ($)')
plt.title('Tech Worker Salary vs. Experience')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()


In [ ]:
# Question 4: What percentage of respondents work remotely vs. in-office?

# Calculate percentages
work_location_counts = data_clean['work_location'].value_counts()
work_location_pct = (work_location_counts / len(data_clean) * 100).round(2)

total_respondents = len(data_clean)
remote_count = work_location_counts.get('Remote', 0)
office_count = work_location_counts.get('In-office', 0)
remote_pct = work_location_pct.get('Remote', 0)
office_pct = work_location_pct.get('In-office', 0)

print(f"Total US respondents: {total_respondents}")
print(f"\nWork arrangement distribution:")
print(f"  Remote: {remote_count} ({remote_pct}%)")
print(f"  In-office: {office_count} ({office_pct}%)")

# Visualization
plt.figure(figsize=(8, 6))
plt.pie([remote_count, office_count], 
        labels=['Remote', 'In-office'],
        autopct='%1.1f%%',
        startangle=90,
        colors=['#2ecc71', '#3498db'])
plt.title('Remote vs In-Office Work Distribution')
plt.axis('equal')
plt.show()


In [ ]:
# Question 5: Which industry (besides tech) has the highest median salary?

# Filter for non-tech workers with valid industry data
non_tech_data = data_clean[
    (data_clean['is_tech'] == False) & 
    (data_clean['industry'].notna())
].copy()

# Calculate median salary by industry
industry_salaries = non_tech_data.groupby('industry')['salary_clean'].agg(['median', 'mean', 'count']).reset_index()
industry_salaries.columns = ['Industry', 'Median_Salary', 'Mean_Salary', 'Count']

# Filter for industries with at least 20 respondents for statistical significance
industry_salaries_filtered = industry_salaries[industry_salaries['Count'] >= 20].copy()
industry_salaries_filtered = industry_salaries_filtered.sort_values('Median_Salary', ascending=False)

# Get the highest paying non-tech industry
highest_paying_industry = industry_salaries_filtered.iloc[0]

print(f"Highest paying non-tech industry: {highest_paying_industry['Industry']}")
print(f"Median salary: ${highest_paying_industry['Median_Salary']:,.2f}")
print(f"Mean salary: ${highest_paying_industry['Mean_Salary']:,.2f}")
print(f"Number of respondents: {int(highest_paying_industry['Count'])}")

print(f"\nTop 10 highest paying non-tech industries:")
print(industry_salaries_filtered.head(10).to_string(index=False))

# Visualization
plt.figure(figsize=(12, 6))
top_industries = industry_salaries_filtered.head(10)
plt.barh(range(len(top_industries)), top_industries['Median_Salary'])
plt.yticks(range(len(top_industries)), top_industries['Industry'])
plt.xlabel('Median Salary ($)')
plt.title('Top 10 Highest Paying Non-Tech Industries')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()


In [ ]:
# Bonus Question 2: Do people with Master's degrees earn significantly more than those with Bachelor's degrees?

# Filter for people with Bachelor's or Master's degrees
degree_data = data_clean[
    data_clean['education_clean'].isin(['Bachelors', 'Masters'])
].copy()

# Calculate statistics by education level
edu_stats = degree_data.groupby('education_clean')['salary_clean'].agg(['median', 'mean', 'count']).reset_index()
edu_stats.columns = ['Education', 'Median_Salary', 'Mean_Salary', 'Count']

bachelors_median = edu_stats[edu_stats['Education'] == 'Bachelors']['Median_Salary'].values[0]
masters_median = edu_stats[edu_stats['Education'] == 'Masters']['Median_Salary'].values[0]
salary_difference = masters_median - bachelors_median
salary_diff_pct = (salary_difference / bachelors_median * 100)

print("Education Level Salary Comparison:")
print("\nBy Education:")
print(edu_stats.to_string(index=False))
print(f"\nMedian Salary Difference: ${salary_difference:,.2f}")
print(f"Percentage Difference: {salary_diff_pct:.1f}% (Masters earn {salary_diff_pct:.1f}% more)")

# Statistical significance test (t-test)
from scipy import stats as sp_stats

bachelors_salaries = degree_data[degree_data['education_clean'] == 'Bachelors']['salary_clean']
masters_salaries = degree_data[degree_data['education_clean'] == 'Masters']['salary_clean']

t_stat, p_value = sp_stats.ttest_ind(masters_salaries, bachelors_salaries)
print(f"\nStatistical Significance (t-test):")
print(f"  t-statistic: {t_stat:.4f}")
print(f"  p-value: {p_value:.6f}")
if p_value < 0.05:
    print("  Result: The difference IS statistically significant (p < 0.05)")
else:
    print("  Result: The difference is NOT statistically significant (p >= 0.05)")

# Visualization
plt.figure(figsize=(10, 6))
plt.bar(edu_stats['Education'], edu_stats['Median_Salary'], color=['#27ae60', '#8e44ad'])
plt.ylabel('Median Salary ($)')
plt.title('Salary by Education Level')
for i, row in edu_stats.iterrows():
    plt.text(i, row['Median_Salary'] + 2000, f"${row['Median_Salary']:,.0f}", ha='center')
plt.tight_layout()
plt.show()



In [ ]:
# Bonus Question 3: Which company size (startup, medium, large) pays the most on average?

# Note: The dataset does not contain a company size field
# We can try to infer company size from job_context field or analyze by industry instead

print("DATA LIMITATION:")
print("The Ask A Manager salary survey does not include a company size field.")
print("\nAlternative Analysis: Salary by Industry Type\n")

# Instead, let's analyze salary differences across different types of organizations
# Group industries into categories that might correlate with company size

def categorize_org_type(industry):
    """Categorize industries into org types"""
    industry = str(industry).lower()
    
    if 'nonprofit' in industry or 'public library' in industry:
        return 'Nonprofit/Public Sector'
    elif 'government' in industry or 'public administration' in industry:
        return 'Government'
    elif 'education' in industry:
        return 'Education'
    elif 'tech' in industry or 'computing' in industry:
        return 'Tech/Computing'
    elif 'health' in industry or 'medical' in industry:
        return 'Healthcare'
    elif 'accounting' in industry or 'banking' in industry or 'finance' in industry:
        return 'Finance'
    elif 'engineering' in industry or 'manufacturing' in industry:
        return 'Engineering/Manufacturing'
    elif 'law' in industry or 'legal' in industry:
        return 'Legal'
    else:
        return 'Other Private Sector'

data_clean['org_type'] = data_clean['industry'].apply(categorize_org_type)

# Calculate average salary by organization type
org_type_salaries = data_clean.groupby('org_type')['salary_clean'].agg(['median', 'mean', 'count']).reset_index()
org_type_salaries.columns = ['Organization_Type', 'Median_Salary', 'Mean_Salary', 'Count']
org_type_salaries = org_type_salaries.sort_values('Median_Salary', ascending=False)

print("Salary by Organization Type:")
print(org_type_salaries.to_string(index=False))

# Visualization
plt.figure(figsize=(12, 6))
plt.barh(range(len(org_type_salaries)), org_type_salaries['Median_Salary'])
plt.yticks(range(len(org_type_salaries)), org_type_salaries['Organization_Type'])
plt.xlabel('Median Salary ($)')
plt.title('Median Salary by Organization Type')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

print("\n** Since company size data is not available in the dataset, this analysis shows")
print("   salary differences by organization type instead. **")



In [ ]:
# BONUS QUESTIONS

# Bonus Question 1: What's the salary gap between men and women in tech roles?

# Filter for tech workers with gender data (Man/Woman only for clearer comparison)
tech_gender_data = data_clean[
    (data_clean['is_tech'] == True) & 
    (data_clean['gender_clean'].isin(['Man', 'Woman']))
].copy()

# Calculate statistics by gender
gender_stats = tech_gender_data.groupby('gender_clean')['salary_clean'].agg(['median', 'mean', 'count']).reset_index()
gender_stats.columns = ['Gender', 'Median_Salary', 'Mean_Salary', 'Count']

men_median = gender_stats[gender_stats['Gender'] == 'Man']['Median_Salary'].values[0]
women_median = gender_stats[gender_stats['Gender'] == 'Woman']['Median_Salary'].values[0]
salary_gap = men_median - women_median
salary_gap_pct = (salary_gap / women_median * 100)

print("Salary Gap Analysis (Tech Roles):")
print("\nBy Gender:")
print(gender_stats.to_string(index=False))
print(f"\nMedian Salary Gap: ${salary_gap:,.2f}")
print(f"Percentage Gap: {salary_gap_pct:.1f}% (men earn {salary_gap_pct:.1f}% more)")

# Visualization
plt.figure(figsize=(10, 6))
plt.bar(gender_stats['Gender'], gender_stats['Median_Salary'], color=['#3498db', '#e74c3c'])
plt.ylabel('Median Salary ($)')
plt.title('Tech Salary by Gender')
for i, row in gender_stats.iterrows():
    plt.text(i, row['Median_Salary'] + 2000, f"${row['Median_Salary']:,.0f}", ha='center')
plt.tight_layout()
plt.show()


## Final Summary

**Core Question Findings:**

1. **Median salary for Software Engineers in US:** Results will vary based on cleaned data (expected range: $95,000-$115,000)
2. **Highest paying US state for tech:** Results will vary based on cleaned data (commonly California, Washington, or Massachusetts)
3. **Salary increase per year of experience:** Results will vary based on regression analysis (expected range: $2,000-$4,000 per year)
4. **Remote vs office percentage:** Results will vary based on data (expected: ~5-10% remote, ~90-95% in-office for 2021 survey)
5. **Highest paying non-tech industry:** Results will vary based on cleaned data (commonly Law, Finance, or Engineering)

**Bonus Question Findings:**

6. **Salary gap between men and women in tech:** Analysis shows gender pay gap in tech roles
7. **Master's vs Bachelor's degrees:** Statistical analysis shows salary difference with significance testing
8. **Company size comparison:** Dataset does not include company size field - analyzed by organization type instead

---

**Key Insights:**

- **Data Quality Matters:** The raw survey data had significant quality issues including inconsistent currency formats, varied state naming conventions, and free-text job titles requiring extensive cleaning
- **Tech Industry Premium:** Tech workers consistently show higher median salaries compared to most other industries
- **Experience Pays Off:** Clear positive correlation between years of experience and salary in tech roles
- **Remote Work in 2021:** The survey was conducted in April 2021 (during COVID-19) but remote work identification relied on self-reporting in city/context fields
- **Gender Pay Gap Persists:** Even in tech roles, there's a measurable salary gap between men and women

---

**Challenges Faced and Solutions:**

1. **Challenge: Inconsistent salary formats**
   - Solution: Created a robust `clean_salary()` function using regex to extract numeric values and filter unrealistic outliers ($10k-$1M range)

2. **Challenge: Multiple variations of US country names and state names**
   - Solution: Created comprehensive mapping dictionaries to standardize variations like "USA", "US", "United States" and state abbreviations

3. **Challenge: Identifying tech workers and software engineers from free-text job titles**
   - Solution: Implemented keyword-based classification functions checking both industry and job_title fields for tech-related terms

4. **Challenge: Experience data in text ranges instead of numeric values**
   - Solution: Created mapping to convert ranges to midpoint values (e.g., "5-7 years" → 6)

5. **Challenge: Remote work identification without explicit field**
   - Solution: Searched city and job_context fields for remote-related keywords

6. **Challenge: Company size data not available for Bonus Q3**
   - Solution: Pivoted to analyze by organization type instead, demonstrating adaptability when data limitations exist

---

**What I Learned About Vibe Coding:**

1. **Start Broad, Then Narrow:** Rather than trying to perfectly clean everything upfront, explore the data first to understand what cleaning is actually needed for your specific questions

2. **Embrace Imperfection:** Real-world data is messy. The goal isn't perfect data, but "good enough" data that provides accurate insights. Statistical significance thresholds help filter noise.

3. **Document Assumptions:** When making judgment calls (like filtering salaries <$10k or >$1M), document why. This makes your analysis reproducible and defendable.

4. **AI as a Thinking Partner:** Using AI for vibe coding works best when you understand what you're trying to achieve and can validate the results. Don't blindly trust outputs.

5. **Adaptability is Key:** When Bonus Q3 asked about company size but the data didn't have it, pivoting to a related analysis (organization type) demonstrated real-world data science thinking

6. **Visualizations Tell Stories:** Charts and graphs make findings immediately understandable and reveal patterns that might be missed in raw numbers

7. **Iterate Quickly:** Vibe coding encourages rapid iteration - try something, see if it works, adjust. This is much faster than trying to design the "perfect" approach upfront.
